In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## やったこと

- テキストのクリーニング処理 -> 改善
- host, categoryのカテゴリカル変数をエンベディングして入力 -> 改善

- epochs=20で、early-stoppingはあまり良くならなかった -> とりあえず速く数を回したいので、epochs=4でやっている


- batch_size=8以上にすると、out_of_memoryになる

- MSELossを使用 -> 悪化
- titleは分けて、別のエンベディングとして入力 -> 悪化



- BERTを2つ使う -> gpu不足
- クラス分類問題にする（30*num_class） -> 学習が安定しない（nan）
- 30個の目的変数それぞれ独立に予測するモデル -> 約30時間必要、あまり精度が出ないように見える -> 関連する目的変数だけをグルーピングしてモデルを分ける必要？

In [0]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os, sys, gc, random, multiprocessing, glob, time

DATA_DIR = '/content/drive/My Drive/Colab Notebooks/GoogleQuest/input/google-quest-challenge'
# DATA_DIR = '../input/google-quest-challenge'
# DATA_DIR = 'D:/project/ICF_AutoCapsule_disabled/kaggle/google-quest-challenge'
# BERT_DIR = 'D:/project/ICF_AutoCapsule_disabled/BERT'

In [0]:
# !pip install ../input/sacremoses/sacremoses-master/
# !pip install ../input/transformers/transformers-master/

In [0]:
!pip install transformers
!pip install flashtext

In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

#from ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

from scipy.stats import spearmanr

import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)

from tqdm import tqdm
print(transformers.__version__)

2.4.1


In [0]:
## Make results reproducible .Else noone will believe you .
import random

def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [0]:
class PipeLineConfig:
    def __init__(self, lr, warmup, epochs, patience, batch_size, seed, name, question_weight,answer_weight,fold,train):
        self.lr = lr
        self.warmup = warmup
        self.epochs = epochs
        self.patience = patience
        self.batch_size = batch_size
        self.seed = seed
        self.name = name
        self.question_weight = question_weight
        self.answer_weight =answer_weight
        self.fold = fold
        self.train = train

In [0]:
config = PipeLineConfig(lr=1e-5, \
                        warmup=0.01, \
                        epochs=20, \
                        patience=3, \
                        batch_size=8, \
                        seed=42, \
                        name='reModel', \
                        question_weight=0.5, \
                        answer_weight=0.5, \
                        fold=5, \
                        train=True
                       )

In [0]:
seed_everything(config.seed)

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cuda


In [0]:
sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [0]:
target_columns = sub.columns.values[1:].tolist()
target_columns

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [0]:
train = pd.read_csv(f'{DATA_DIR}/train.csv')
train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [0]:
test = pd.read_csv(f'{DATA_DIR}/test.csv')
test.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


## Preprocessing

In [0]:
import re
from flashtext import KeywordProcessor

In [0]:
PUNCTS = {
            '》', '〞', '¢', '‹', '╦', '║', '♪', 'Ø', '╩', '\\', '★', '＋', 'ï', '<', '?', '％', '+', '„', 'α', '*', '〰', '｟', '¹', '●', '〗', ']', '▾', '■', '〙', '↓', '´', '【', 'ᴵ',
            '"', '）', '｀', '│', '¤', '²', '‡', '¿', '–', '」', '╔', '〾', '%', '¾', '←', '〔', '＿', '’', '-', ':', '‧', '｛', 'β', '（', '─', 'à', 'â', '､', '•', '；', '☆', '／', 'π',
            'é', '╗', '＾', '▪', ',', '►', '/', '〚', '¶', '♦', '™', '}', '″', '＂', '『', '▬', '±', '«', '“', '÷', '×', '^', '!', '╣', '▲', '・', '░', '′', '〝', '‛', '√', ';', '】', '▼',
            '.', '~', '`', '。', 'ə', '］', '，', '{', '～', '！', '†', '‘', '﹏', '═', '｣', '〕', '〜', '＼', '▒', '＄', '♥', '〛', '≤', '∞', '_', '[', '＆', '→', '»', '－', '＝', '§', '⋅', 
            '▓', '&', 'Â', '＞', '〃', '|', '¦', '—', '╚', '〖', '―', '¸', '³', '®', '｠', '¨', '‟', '＊', '£', '#', 'Ã', "'", '▀', '·', '？', '、', '█', '”', '＃', '⊕', '=', '〟', '½', '』',
            '［', '$', ')', 'θ', '@', '›', '＠', '｝', '¬', '…', '¼', '：', '¥', '❤', '€', '−', '＜', '(', '〘', '▄', '＇', '>', '₤', '₹', '∅', 'è', '〿', '「', '©', '｢', '∙', '°', '｜', '¡', 
            '↑', 'º', '¯', '♫', '#'
          }


mispell_dict = {"aren't" : "are not", "can't" : "cannot", "couldn't" : "could not",
"couldnt" : "could not", "didn't" : "did not", "doesn't" : "does not",
"doesnt" : "does not", "don't" : "do not", "hadn't" : "had not", "hasn't" : "has not",
"haven't" : "have not", "havent" : "have not", "he'd" : "he would", "he'll" : "he will", "he's" : "he is", "i'd" : "I would",
"i'd" : "I had", "i'll" : "I will", "i'm" : "I am", "isn't" : "is not", "it's" : "it is",
"it'll":"it will", "i've" : "I have", "let's" : "let us", "mightn't" : "might not", "mustn't" : "must not", 
"shan't" : "shall not", "she'd" : "she would", "she'll" : "she will", "she's" : "she is", "shouldn't" : "should not", "shouldnt" : "should not",
"that's" : "that is", "thats" : "that is", "there's" : "there is", "theres" : "there is", "they'd" : "they would", "they'll" : "they will",
"they're" : "they are", "theyre":  "they are", "they've" : "they have", "we'd" : "we would", "we're" : "we are", "weren't" : "were not",
"we've" : "we have", "what'll" : "what will", "what're" : "what are", "what's" : "what is", "what've" : "what have", "where's" : "where is",
"who'd" : "who would", "who'll" : "who will", "who're" : "who are", "who's" : "who is", "who've" : "who have", "won't" : "will not", "wouldn't" : "would not", "you'd" : "you would",
"you'll" : "you will", "you're" : "you are", "you've" : "you have", "'re": " are", "wasn't": "was not", "we'll":" will", "didn't": "did not", "tryin'":"trying"}


kp = KeywordProcessor(case_sensitive=True)
for k, v in mispell_dict.items():
    kp.add_keyword(k, v)

def clean_punct(text):
    text = str(text)
    for punct in PUNCTS:
        text = text.replace(punct, ' {} '.format(punct))
    return text


def preprocessing(text):
    text = text.lower()
    text = re.sub(r'(\&lt)|(\&gt)', ' ', text)
    
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' url ', text)
    text = kp.replace_keywords(text)
    text = clean_punct(text)
    text = re.sub(r'\n\r', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    
    return text

In [0]:
category_list = ['CULTURE', 'LIFE_ARTS', 'SCIENCE', 'STACKOVERFLOW', 'TECHNOLOGY']

host_list = ['academia.stackexchange.com', 'android.stackexchange.com',
       'anime.stackexchange.com', 'apple.stackexchange.com',
       'askubuntu.com', 'bicycles.stackexchange.com',
       'biology.stackexchange.com', 'blender.stackexchange.com',
       'boardgames.stackexchange.com', 'chemistry.stackexchange.com',
       'christianity.stackexchange.com', 'codereview.stackexchange.com',
       'cooking.stackexchange.com', 'crypto.stackexchange.com',
       'cs.stackexchange.com', 'dba.stackexchange.com',
       'diy.stackexchange.com', 'drupal.stackexchange.com',
       'dsp.stackexchange.com', 'electronics.stackexchange.com',
       'ell.stackexchange.com', 'english.stackexchange.com',
       'expressionengine.stackexchange.com', 'gamedev.stackexchange.com',
       'gaming.stackexchange.com', 'gis.stackexchange.com',
       'graphicdesign.stackexchange.com', 'judaism.stackexchange.com',
       'magento.stackexchange.com', 'math.stackexchange.com',
       'mathematica.stackexchange.com', 'mathoverflow.net',
       'mechanics.stackexchange.com', 'meta.askubuntu.com',
       'meta.christianity.stackexchange.com',
       'meta.codereview.stackexchange.com', 'meta.math.stackexchange.com',
       'meta.stackexchange.com', 'meta.superuser.com',
       'money.stackexchange.com', 'movies.stackexchange.com',
       'music.stackexchange.com', 'photo.stackexchange.com',
       'physics.stackexchange.com', 'programmers.stackexchange.com',
       'raspberrypi.stackexchange.com', 'robotics.stackexchange.com',
       'rpg.stackexchange.com', 'salesforce.stackexchange.com',
       'scifi.stackexchange.com', 'security.stackexchange.com',
       'serverfault.com', 'sharepoint.stackexchange.com',
       'softwarerecs.stackexchange.com', 'stackoverflow.com',
       'stats.stackexchange.com', 'superuser.com',
       'tex.stackexchange.com', 'travel.stackexchange.com',
       'unix.stackexchange.com', 'ux.stackexchange.com',
       'webapps.stackexchange.com', 'webmasters.stackexchange.com',
       'wordpress.stackexchange.com']

## Dataset

In [0]:
MAX_LEN = 512
#MAX_Q_LEN = 250
#MAX_A_LEN = 259
SEP_TOKEN_ID = 102 # bert-base-uncasedにおけるvocabの'[SEP']が、102番目という意味

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True):
        self.df = df
        self.train_mode = train_mode
        self.labeled = labeled
        #self.tokenizer = BertTokenizer.from_pretrained(BERT_DIR+'/bert-base-uncased')
        #self.tokenizer = BertTokenizer.from_pretrained('../input/bert-base-uncased/')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __getitem__(self, index):
        """
        token_id列
        segment_id列
        label列
        """
        row = self.df.iloc[index]
        token_ids, seg_ids = self.get_token_ids(row)
        if self.labeled:
            labels = self.get_label(row)
            return token_ids, seg_ids, torch.tensor(row.category), torch.tensor(row.host), labels
        else:
            return token_ids, seg_ids, torch.tensor(row.category), torch.tensor(row.host)

    def __len__(self):
        return len(self.df)


#     def select_tokens(self, tokens, max_num):
#         if len(tokens) <= max_num:
#             return tokens
#         if self.train_mode:
#             num_remove = len(tokens) - max_num
#             remove_start = random.randint(0, len(tokens)-num_remove-1)
#             return tokens[:remove_start] + tokens[remove_start + num_remove:]
#         else:
#             return tokens[:max_num//2] + tokens[-(max_num - max_num//2):]

    def trim_input(self, title, question, answer, max_sequence_length=MAX_LEN, 
                t_max_len=30, q_max_len=239, a_max_len=239):
        """
        title. question, answerそれぞれのセンテンスを、tokenizeする
        max_lengthに足りない分は、
        """
        t = self.tokenizer.tokenize(title)
        q = self.tokenizer.tokenize(question)
        a = self.tokenizer.tokenize(answer)

        t_len = len(t)
        q_len = len(q)
        a_len = len(a)

        if (t_len+q_len+a_len+4) > max_sequence_length:

            if t_max_len > t_len:
                """
                titleが短い場合、
                最大長に足りない長さを、半分ずつqとaに加える
                """
                t_new_len = t_len
                a_max_len = a_max_len + math.floor((t_max_len - t_len)/2) # 切り捨て
                q_max_len = q_max_len + math.ceil((t_max_len - t_len)/2) # 切り上げ
            else:
                """
                titleが長い場合、最大長で切る
                """
                t_new_len = t_max_len

            if a_max_len > a_len:
                """
                answerに加えても短い場合、
                最大長に足りない長さを、qに加える
                """
                a_new_len = a_len 
                q_new_len = q_max_len + (a_max_len - a_len)
            elif q_max_len > q_len:
                a_new_len = a_max_len + (q_max_len - q_len)
                q_new_len = q_len
            else:
                a_new_len = a_max_len
                q_new_len = q_max_len


            if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
                raise ValueError("New sequence length should be %d, but is %d" 
                                 % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))

            t = t[:t_new_len]
            q = q[:q_new_len]
            a = a[:a_new_len]

        return t, q, a
        
    def get_token_ids(self, row):
        t_tokens, q_tokens, a_tokens = self.trim_input(row.question_title, row.question_body, row.answer)
        
        # BERTの入力タイプに変換([CLS]と[SEP]をつないで、１つのsetentenceに)
        tokens = ['[CLS]'] + t_tokens + ['[SEP]'] + q_tokens + ['[SEP]'] + a_tokens + ['[SEP]']
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        if len(token_ids) < MAX_LEN:
            """0で後ろからpadding"""
            token_ids += [0] * (MAX_LEN - len(token_ids))
        ids = torch.tensor(token_ids)
        seg_ids = self.get_seg_ids(ids)  # segment_embを区別するindex
        return ids, seg_ids
    
    def get_seg_ids(self, ids):
        """
        いくつめの文かを区別するsegment_idを、各文字に振る
        """
        seg_ids = torch.zeros_like(ids) # [max_len]のtorch_tensor
        seg_idx = 0
        first_sep = True
        for i, e in enumerate(ids):
            seg_ids[i] = seg_idx
            if e == SEP_TOKEN_ID: # [SEP]の場合
                if first_sep:
                    first_sep = False
                else:
                    seg_idx = 1
        pad_idx = torch.nonzero(ids == 0)  # bert-base_uncasedのvocabで、[PAD]は0番目であるので、PADの部分のindexだけ抽出
        seg_ids[pad_idx] = 0

        return seg_ids

    def get_label(self, row):
        #print(row[target_columns].values)
        return torch.tensor(row[target_columns].values.astype(np.float32))

    def collate_fn(self, batch):
        """
        labelデータを持つモードと、ない完全な推論モードでは、batchのshapeが異なるので(labelが2番目の要素にあるなし)
        """
        token_ids = torch.stack([x[0] for x in batch])
        seg_ids = torch.stack([x[1] for x in batch])
        category = torch.stack([x[2] for x in batch])
        host = torch.stack([x[3] for x in batch])
    
        if self.labeled:
            labels = torch.stack([x[-1] for x in batch])
            return token_ids, seg_ids, category, host, labels
        else:
            return token_ids, seg_ids, category, host

In [0]:
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0):
    df = pd.read_csv(f'{DATA_DIR}/train.csv')

    # cleaning
    df['question_title'] = df['question_title'].apply(lambda x : preprocessing(x))
    df['question_body'] = df['question_body'].apply(lambda x : preprocessing(x))
    df['answer'] = df['answer'].apply(lambda x : preprocessing(x))
    
    # label encode
    le_category = LabelEncoder()
    le_category.fit(category_list)
    for c in set(df.category):
        if c not in category_list:
            df.category = df.category.replace(c, np.nan)
            df.category = df.category.fillna(train.category.mode()[0])
    df.category = le_category.transform(df.category)

    
    le_host = LabelEncoder()
    le_host.fit(host_list)
    for c in set(df.host):
        if c not in host_list:
            df.host = df.host.replace(c, np.nan)
            df.host = df.host.fillna(train.host.mode()[0])
    df.host = le_host.transform(df.host)


    df = shuffle(df, random_state=1234)
    gkf = GroupKFold(n_splits=5).split(X=df.question_body, groups=df.question_body)
    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold == ifold:
            df_train = df.iloc[train_idx]
            df_val = df.iloc[valid_idx]
            break

    print('train', df_train.shape)
    print('val', df_val.shape)

    ds_train = QuestDataset(df_train, train_mode=True)
    train_loader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=ds_train.collate_fn, drop_last=True)
    train_loader.num = len(df_train)

    ds_val = QuestDataset(df_val, train_mode=False)
    val_loader = torch.utils.data.DataLoader(ds_val, batch_size=val_batch_size, shuffle=False, num_workers=0, collate_fn=ds_val.collate_fn, drop_last=False)
    val_loader.num = len(df_val)
    val_loader.df = df_val

    return train_loader, val_loader, df_val.shape[0], valid_idx


def get_test_loader(batch_size=4):
    df = pd.read_csv(f'{DATA_DIR}/test.csv')

    # cleaning
    df['question_title'] = df['question_title'].apply(lambda x : preprocessing(x))
    df['question_body'] = df['question_body'].apply(lambda x : preprocessing(x))
    df['answer'] = df['answer'].apply(lambda x : preprocessing(x))
    
    # label encode
    le_category = LabelEncoder()
    le_category.fit(category_list)
    for c in set(df.category):
        if c not in category_list:
            df.category = df.category.replace(c, np.nan)
            df.category = df.category.fillna(train.category.mode()[0])
    df.category = le_category.transform(df.category)
    
    le_host = LabelEncoder()
    le_host.fit(host_list)
    for c in set(df.host):
        if c not in host_list:
            df.host = df.host.replace(c, np.nan)
            df.host = df.host.fillna(train.host.mode()[0])
    df.host = le_host.transform(df.host)


    ds_test = QuestDataset(df, train_mode=False, labeled=False)
    loader = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=ds_test.collate_fn, drop_last=False)
    loader.num = len(df)
    
    return loader

In [0]:
class QuestModel(nn.Module):
    def __init__(self, n_classes=30):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
        #self.bert_model = BertModel.from_pretrained(BERT_DIR+'/bert-base-uncased/')
        #self.bert_model = BertModel.from_pretrained('../input/bert-base-uncased/')
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        
        self.emb_category = nn.Embedding(len(category_list), 3)
        self.emb_host = nn.Embedding(len(host_list), 5)
        
        self.fc = nn.Linear(768+3+5, n_classes)

    def forward(self, ids, seg_ids, category, host):
        attention_mask = (ids > 0)  # ids==0([PAD])部分だけFalseとなるので、そこだけattention_weightを0に
        layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=seg_ids, attention_mask=attention_mask)
        #print(layers.size())  # (batch_size,sequence_length, 768)
        #print(pool_out.size())  # (batch_size, 768), first token of last layerをいじったもの
        
        out = F.avg_pool1d(layers.transpose(1,2), kernel_size=layers.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        
        emb_category = self.emb_category(category)
        emb_host = self.emb_host(host)
        
        #print(out.shape)
        #print(emb_category.shape)
        
        out = torch.cat([out, emb_category], dim=-1)
        out = torch.cat([out, emb_host], dim=-1)
        
        #print(out.shape)
        
        out = F.dropout(out, p=0.2, training=self.training)
        
#         out = F.dropout(layers[-1][:, 0, :], p=0.2, training=self.training)
#         out =  F.dropout(pool_out, p=0.2, training=self.training)
        logit = self.fc(out)
        return logit # 単に30種類の出力値を算出
    

In [0]:
def train_model(train_loader, optimizer, criterion, scheduler):
    model.train()
    avg_loss = 0.    
    for idx, batch in enumerate(tqdm(train_loader)):
        ids_train, seg_ids_train, category_train, host_train, label_ids_train = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device), batch[4].to(device)
        
        #print(host_train)
        
        logits = model(ids_train, seg_ids_train, category_train, host_train)
        #logits = torch.sigmoid(model(ids_train, seg_ids_train))
        
        loss = config.question_weight*criterion(logits[:,0:21], label_ids_train[:,0:21]) + config.answer_weight*criterion(logits[:,21:30], label_ids_train[:,21:30])
        
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        del ids_train, seg_ids_train, label_ids_train

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

def val_model(val_loader, val_length, batch_size=8):
    model.eval() # eval mode  
    avg_val_loss = 0.
    
    valid_preds = np.zeros((val_length, len(target_columns)))
    original = np.zeros((val_length, len(target_columns)))
    
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(val_loader)):
            ids_val, seg_ids_val, category_val, host_val, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device), batch[4].to(device)
            
            logits = torch.sigmoid(model(ids_val, seg_ids_val, category_val, host_val))
            
            avg_val_loss += criterion(logits, labels).item() / len(val_loader)
            valid_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
        
        score = 0
        preds = torch.tensor(valid_preds).numpy()
        #preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()
        
        rho_val = np.mean([spearmanr(original[:, i], preds[:,i]).correlation for i in range(preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        for i in range(len(target_columns)):
            print(i, spearmanr(original[:,i], preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], preds[:, i]).correlation)
    
    return avg_val_loss, score/len(target_columns), preds, original

In [0]:
def calc_spearman(preds, targets):
    score = 0
    for i in range(targets.shape[1]):
        score += np.nan_to_num(spearmanr(targets[:, i], preds[:, i]).correlation)
    return score/targets.shape[1]

In [23]:
cv_list = []
for fold in range(config.fold):
    print('---%d-Fold---'%(fold+1))
    
    patience = 0
    best_loss   = 100.0
    best_score      = -1.
    best_preds = 0
    best_param_loss = None
    best_param_score = None

    model = QuestModel(n_classes=30).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, eps=4e-5)
    criterion = nn.BCEWithLogitsLoss()
    
    for epoch in range(config.epochs):
        
        torch.cuda.empty_cache()
        start_time   = time.time()
        
        train_loader, val_loader, val_length, val_idx = get_train_val_loaders(batch_size=config.batch_size, val_batch_size=config.batch_size, ifold=fold)
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup, num_training_steps=config.epochs*len(train_loader))
        
        loss_train = train_model(train_loader, optimizer, criterion, scheduler)
        loss_val, score_val, preds, original = val_model(val_loader, val_length, batch_size=config.batch_size)
        print(f'Epoch {(epoch+1)}, train_loss: {loss_train}, val_loss: {loss_val}, score_val: {score_val}, time: {(time.time()-start_time)}')

        if score_val > best_score:
            best_score = score_val
            best_param_score = model.state_dict()
            print('best_param_score_{}_{}.pt'.format(config.name ,fold+1))
            torch.save(best_param_score, '/content/drive/My Drive/Colab Notebooks/GoogleQuest/best_param_score_{}_{}.pt'.format(config.name ,fold+1))
        else:
            patience += 1
            if patience >= config.patience:
                del train_loader, val_loader, loss_train, loss_val, score_val, preds
                torch.cuda.empty_cache()
                gc.collect()
                break
    
        del train_loader, val_loader, loss_train, loss_val, score_val, preds
        torch.cuda.empty_cache()
        gc.collect()
        
    model.load_state_dict(best_param_score)
    print('best_param_score_{}_{}.pt'.format(config.name ,fold+1))
    torch.save(best_param_score, '/content/drive/My Drive/Colab Notebooks/GoogleQuest/best_param_score_{}_{}.pt'.format(config.name ,fold+1))   
    cv_list.append(best_score)

    torch.cuda.empty_cache()
    gc.collect()
    
print('CV_score: ', np.mean(cv_list))

---1-Fold---
train (4863, 41)
val (1216, 41)


100%|██████████| 152/152 [00:41<00:00,  3.74it/s]


 val_spearman-rho: 0.31514                                                                                                    
0 SpearmanrResult(correlation=0.23992234420203826, pvalue=2.214349186104115e-17)
1 SpearmanrResult(correlation=0.4424680267402274, pvalue=1.886342458615563e-59)
2 SpearmanrResult(correlation=0.3897295951786787, pvalue=2.176424458590335e-45)
3 SpearmanrResult(correlation=0.20812984863299508, pvalue=2.289675050886265e-13)
4 SpearmanrResult(correlation=0.25493687334957316, pvalue=1.709972825083329e-19)
5 SpearmanrResult(correlation=0.36801005951569116, pvalue=2.684731392415747e-40)
6 SpearmanrResult(correlation=0.28226937769801874, pvalue=1.040109288311884e-23)
7 SpearmanrResult(correlation=0.44563405034998993, pvalue=2.2349482453181378e-60)
8 SpearmanrResult(correlation=0.35108699051707115, pvalue=1.3607763989781864e-36)
9 SpearmanrResult(correlation=-0.010921467854678166, pvalue=0.7036008176580273)
10 SpearmanrResult(correlation=0.3653579041915821, pvalue=1.0577

100%|██████████| 152/152 [00:41<00:00,  3.66it/s]


 val_spearman-rho: 0.36109                                                                                                    
0 SpearmanrResult(correlation=0.3147817063495091, pvalue=2.2432739580816492e-29)
1 SpearmanrResult(correlation=0.5395938151647794, pvalue=8.110065998097614e-93)
2 SpearmanrResult(correlation=0.39427402773591463, pvalue=1.6710682504685977e-46)
3 SpearmanrResult(correlation=0.27827223390282896, pvalue=4.614817379915664e-23)
4 SpearmanrResult(correlation=0.34708563012201926, pvalue=9.491903818650647e-36)
5 SpearmanrResult(correlation=0.4005671801097371, pvalue=4.4689525187350414e-48)
6 SpearmanrResult(correlation=0.3070517489916053, pvalue=5.811259380991265e-28)
7 SpearmanrResult(correlation=0.4528031481169679, pvalue=1.6410729721641586e-62)
8 SpearmanrResult(correlation=0.5001146409702214, pvalue=6.046326104690645e-78)
9 SpearmanrResult(correlation=-0.03568332249674021, pvalue=0.21370711998957428)
10 SpearmanrResult(correlation=0.4501265013326506, pvalue=1.042078

...

 val_spearman-rho: 0.35707                                                                                                    
0 SpearmanrResult(correlation=0.32394642833458753, pvalue=4.409599523826187e-31)
1 SpearmanrResult(correlation=0.544505884767803, pvalue=9.85703741204326e-95)
2 SpearmanrResult(correlation=0.38388537225986497, pvalue=6.034043271546652e-44)
3 SpearmanrResult(correlation=0.20886642602784788, pvalue=1.9208882938818105e-13)
4 SpearmanrResult(correlation=0.2671717689591384, pvalue=2.643325849562075e-21)
5 SpearmanrResult(correlation=0.4177970603635901, pvalue=1.6174650057300538e-52)
6 SpearmanrResult(correlation=0.3569928294454091, pvalue=7.870912142541467e-38)
7 SpearmanrResult(correlation=0.4978065383775518, pvalue=4.507866075169526e-77)
8 SpearmanrResult(correlation=0.5055358545223284, pvalue=8.270781538236121e-80)
9 SpearmanrResult(correlation=0.07196441520086916, pvalue=0.01210307651057114)
10 SpearmanrResult(correlation=0.4117816283522921, pvalue=6.34857114836

100%|██████████| 152/152 [00:40<00:00,  3.79it/s]


 val_spearman-rho: 0.35922                                                                                                    
0 SpearmanrResult(correlation=0.3238101183738096, pvalue=4.683404784377979e-31)
1 SpearmanrResult(correlation=0.5301933501971424, pvalue=5.037140040432228e-89)
2 SpearmanrResult(correlation=0.3818244158992206, pvalue=1.8620157205304184e-43)
3 SpearmanrResult(correlation=0.20820975730760943, pvalue=2.292144772071029e-13)
4 SpearmanrResult(correlation=0.2722113597756893, pvalue=4.385824319096258e-22)
5 SpearmanrResult(correlation=0.41167101404576617, pvalue=6.787045040006424e-51)
6 SpearmanrResult(correlation=0.34451554017501435, pvalue=3.470289523701544e-35)
7 SpearmanrResult(correlation=0.4951552269671652, pvalue=3.771904545044261e-76)
8 SpearmanrResult(correlation=0.5066962408120939, pvalue=3.166885944501746e-80)
9 SpearmanrResult(correlation=0.08364219753084642, pvalue=0.003527180194736272)
10 SpearmanrResult(correlation=0.42321083752187993, pvalue=5.57701389

100%|██████████| 152/152 [00:40<00:00,  3.84it/s]


 val_spearman-rho: 0.35174                                                                                                    
0 SpearmanrResult(correlation=0.2978000330467524, pvalue=2.638724048749872e-26)
1 SpearmanrResult(correlation=0.5319655170245087, pvalue=1.0236816188510579e-89)
2 SpearmanrResult(correlation=0.38537268120879975, pvalue=2.6623245322773103e-44)
3 SpearmanrResult(correlation=0.19525768237213637, pvalue=6.637719705690363e-12)
4 SpearmanrResult(correlation=0.2564875379431824, pvalue=1.0511931143782057e-19)
5 SpearmanrResult(correlation=0.4146290760549249, pvalue=1.1279005256661904e-51)
6 SpearmanrResult(correlation=0.33740854971127904, pvalue=9.85809791448149e-34)
7 SpearmanrResult(correlation=0.48401653896270896, pvalue=2.3179132416449407e-72)
8 SpearmanrResult(correlation=0.5047683248572652, pvalue=1.557504649843252e-79)
9 SpearmanrResult(correlation=0.10046216132863242, pvalue=0.0004530770692737843)
10 SpearmanrResult(correlation=0.4131600995062155, pvalue=2.7561